In [1]:
import os
import shutil
from backend.database import Database
from backend.face_detector import FaceDetector
from backend.embedding_generator import generate_embedding
from PIL import Image
import numpy as np

/home/danil/Documents/Project/MemeFaceMatcher/meme-comparator/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db = Database(path='../meme_storage/db.faiss')
fd = FaceDetector()

No FAISS index was found. Initializing a new one!
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/w600k_r50.onnx rec

In [3]:
originals = sorted(os.listdir('../meme_storage'))
indexed = []
originals

['m1.jpeg',
 'm10.jpeg',
 'm11.jpeg',
 'm12.jpeg',
 'm2.jpeg',
 'm3.jpeg',
 'm4.jpeg',
 'm5.jpeg',
 'm6.jpeg',
 'm7.jpeg',
 'm8.jpeg',
 'm9.jpeg']

In [4]:
for file in originals:
    print(file)

    img = np.asarray(Image.open('../meme_storage/' + file))

    face_data = fd.detect(img)
    if face_data is None:
        indexed.append(None)
        continue
    else:
        cropped, _ = face_data

    emb = generate_embedding(cropped)

    indexed.append(db.add_vector(emb)[0])

m1.jpeg


/home/danil/Documents/Project/MemeFaceMatcher/meme-comparator/.venv/lib/python3.11/site-packages/insightface/utils/face_align.py:23: FutureWarning: `estimate` is deprecated since version 0.26 and will be removed in version 2.2. Please use `SimilarityTransform.from_estimate` class constructor instead.
  tform.estimate(lmk, dst)


m10.jpeg
m11.jpeg
m12.jpeg
m2.jpeg
No faces were found!
m3.jpeg
m4.jpeg
m5.jpeg
m6.jpeg
No faces were found!
m7.jpeg
m8.jpeg
m9.jpeg


In [5]:
indexed

[np.int64(0),
 np.int64(1),
 np.int64(2),
 np.int64(3),
 None,
 np.int64(4),
 np.int64(5),
 np.int64(6),
 None,
 np.int64(7),
 np.int64(8),
 np.int64(9)]

In [6]:
for original_path, id in zip(originals, indexed):
    if id is None:
        continue

    shutil.copy('../meme_storage/' + original_path, '../meme_storage/' + str(id) + '.jpeg')

In [ ]:
del db # to save the index